In [2]:
from platform import python_version

print('python version of this jupyter notebook:', python_version())

python version of this jupyter notebook: 3.7.6


In [11]:
import pandas as pd
from tqdm.notebook import tqdm
from collections import defaultdict

In [3]:
movies = pd.read_csv('IMDb movies.csv')

/Users/markzuckerberg/anaconda3/envs/wintertime/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print('the length of the dataset is:', len(movies))

the length of the dataset is: 85855


In [5]:
# use 70000 of the dataset as the training set and the rest as the testing set
train_set = movies[:70000]
test_set = movies[70000:]

In [6]:
train_set.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [7]:
actor_movies =  {}
actor_count = {}
vote_add = 0
for index, row in tqdm(train_set.iterrows()):
    if type(row['actors']) != str:
        continue
    actors = row['actors'].split(',')
    for a in actors:
        if a not in actor_movies:
            actor_movies[a] = {row['imdb_title_id']}
            actor_count[a] = {0}
        else:
            actor_movies[a].add(row['imdb_title_id'])
            actor_count[a] 
    vote_add += row['avg_vote']
    
vote_mean = vote_add/len(train_set)

In [35]:
# count the number of movies an actor plays
actor_count = defaultdict(int)
actor_rate = defaultdict(int)

for index, row in tqdm(train_set.iterrows()):
    if type(row['actors']) != str:
        continue
    actors = row['actors'].split(',')
    for a in actors:
        a = a.replace(" ", "")
        actor_count[a] += 1
        actor_rate[a] += row['avg_vote']
        if a == "Angibell": print(row['title'], row['avg_vote'])

Lejos de Casa pelicula Venezolana 9.8



In [30]:
for a in actor_rate:
    actor_rate[a] = actor_rate[a]/actor_count[a]
    
actor_rate = sorted(actor_rate.items(), key= lambda x: x[1], reverse=True)

In [59]:
# global count
# count = [0] * 10
def feata(d):
#     global count
    year = [0] * 11
    movie_year = d['year']
#     print(movie_year)
    if int(movie_year) <= 1930:
        year[0] = 1
#         count[0] += 1
    else:
        one_hot = int((int(movie_year) - 1921)/10)
        year[one_hot] = 1
#         count[one_hot] += 1
        
    return year[:-1]

In [60]:
X = [feata(row) for index, row in tqdm(train_set.iterrows())]
print(count)


[1057, 2840, 3014, 4144, 5151, 6192, 7432, 10159, 18684, 11327]


In [105]:
years = []
for index, row in tqdm(train_set.iterrows()):
    t = [0] * 10
    year = row['year']
    i = 9
    if int(year) <= 1930:
        i = 1
    else: 
        i = int((int(year) - 1921)/10)
    t[i] = 1
    years.append(t)   

In [66]:
genres = []
tmp = set()
for index, row in train_set.iterrows():
    genres.append([])
    for t in row['genre'].split(', '):
        tmp.add(t)
        genres[-1].append(t)
numGenre = len(tmp)
genreToIndex = dict()
for i, g in enumerate(list(tmp)):
    genreToIndex[g] = i
for i, genre in enumerate(genres):
    t = [0] * numGenre
    for g in genre:
        t[genreToIndex[g]] = 1
    genres[i] = t
    
print(genres)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [93]:
countries = []
tmp = set()
frequency = defaultdict(int)
for index, row in train_set.iterrows():
    countries.append([])
#     print(index, row['country'], row['title'])
    if type(row['country']) != str: continue
    for t in row['country'].split(', '):
        tmp.add(t)
        frequency[t] += 1
        countries[-1].append(t)
numCountries = len(tmp)
frequency = sorted(frequency.keys(), key=lambda x: frequency[x], reverse=True)[:10]
print(frequency)
countryToIndex = dict()
for i, c in enumerate(frequency):
    countryToIndex[c] = i
for i, country in enumerate(countries):
    t = [0] * 10
    for c in country:
        if c in frequency:
            t[countryToIndex[c]] = 1
    countries[i] = t
    
# print(countries)

['USA', 'France', 'UK', 'Italy', 'India', 'Japan', 'Germany', 'Canada', 'Spain', 'Hong Kong']


In [96]:
print(countries[2000])
for index, row in train_set.iterrows():
    if index == 2000: 
        print(row['country'])
        break

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
USA


In [103]:
directors = []
tmp = set()
frequency = defaultdict(int)
totalRating = defaultdict(float)
for index, row in train_set.iterrows():
    directors.append([])
#     print(index, row['country'], row['title'])
    if type(row['director']) != str: continue
    for t in row['director'].split(', '):
        tmp.add(t)
        frequency[t] += 1
        totalRating[t] += row['avg_vote']
        directors[-1].append(t)
numDirector = len(tmp)
frequency = sorted(frequency.keys(), key=lambda x: totalRating[x]/frequency[x] if frequency[x] > 20 else 0, reverse=True)[:20]
print(frequency)
directorToIndex = dict()
for i, c in enumerate(frequency):
    directorToIndex[c] = i
for i, director in enumerate(directors):
    t = [0] * 20
    for c in director:
        if c in frequency:
            t[directorToIndex[c]] = 1
    directors[i] = t
    
# print(countries)

['Satyajit Ray', 'Ertem Egilmez', 'Martin Scorsese', 'Akira Kurosawa', 'K. Balachander', 'Billy Wilder', 'Hrishikesh Mukherjee', 'Yasujirô Ozu', 'Steven Spielberg', 'William Wyler', 'Federico Fellini', 'Ingmar Bergman', 'Éric Rohmer', 'Kenji Misumi', 'Luis Buñuel', 'Roman Polanski', 'Rainer Werner Fassbinder', 'Kenji Mizoguchi', 'Mikio Naruse', 'Sergiu Nicolaescu']
